# Librerías necesarias

In [2]:
#!pip -q install -U transformers accelerate
#!pip install kagglehub
#!pip install safetensors
#!pip install opencv-python

# Imports

In [4]:
import os, sys, glob, json, time, random, platform
from pathlib import Path
from contextlib import nullcontext
import warnings

import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
import cv2
from io import BytesIO
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision.models.segmentation import deeplabv3_resnet50
from torchvision.models import ResNet50_Weights

from transformers import (
    SegformerImageProcessor,
    SegformerForSemanticSegmentation,
    get_cosine_schedule_with_warmup
)

from tqdm.auto import tqdm
import kagglehub

# Config

In [6]:
PROJECT_DIR = Path(r"C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias")
OUTPUT_DIR  = PROJECT_DIR / "outputs"
SPLITS_DIR  = OUTPUT_DIR / "splits"
CKPT_DIR    = OUTPUT_DIR / "checkpoints"
LOG_DIR     = OUTPUT_DIR / "logs"
FIG_DIR     = OUTPUT_DIR / "figures"
TABLE_DIR   = OUTPUT_DIR / "tables"
STATE_PATH  = OUTPUT_DIR / "run_state.json"

for d in [OUTPUT_DIR, SPLITS_DIR, CKPT_DIR, LOG_DIR, FIG_DIR, TABLE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [7]:
SEED = 123

# Cityscapes training resolution
RESIZE = (512, 1024)  # (H, W)
CROP   = (512, 1024)

NUM_CLASSES  = 19
IGNORE_INDEX = 255

# Baseline CNN training hyperparams
CNN_EPOCHS    = 30
CNN_BATCH     = 4
CNN_LR        = 3e-4
CNN_WD        = 1e-4
CNN_PATIENCE  = 8

# SegFormer training hyperparams
SF_EPOCHS     = 30
SF_BATCH      = 2
SF_LR         = 6e-5
SF_WD         = 1e-4
SF_PATIENCE   = 8

# Data-efficiency
DATA_EFF_PCTS = [10, 25, 50, 100]

# Efficiency measurement
EFF_WARMUP_STEPS   = 10
EFF_MEASURE_STEPS  = 50

# Visualization
N_VIS_SAMPLES = 3
VIS_RANDOM_SEED = 999

# Corruptions (Deliverable 3)
CORRUPTIONS = [
    ("gaussian_blur", 3),
    ("motion_blur", 3),
    ("gaussian_noise", 3),
    ("brightness", 3),
    ("contrast", 3),
    ("jpeg", 3),
]

# HuggingFace SegFormer
MODEL_ID = "nvidia/segformer-b0-finetuned-cityscapes-512-1024"

# Seed

In [9]:
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)
torch.backends.cudnn.benchmark = True

# Disponibilidad de recursos

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [12]:
PIN_MEMORY = (device.type == "cuda")
# CPU_COUNT = os.cpu_count() or 4
# NUM_WORKERS = min(8, max(2, CPU_COUNT // 2)) if platform.system().lower() == "windows" else 0

IN_NOTEBOOK = ("ipykernel" in sys.modules) or ("JPY_PARENT_PID" in os.environ)
if platform.system().lower() == "windows" and IN_NOTEBOOK:
    # Prevent silent deadlock in Jupyter on Windows
    NUM_WORKERS = 0
else:
    CPU_COUNT = os.cpu_count() or 4
    NUM_WORKERS = min(8, max(2, CPU_COUNT // 2))

print("PIN_MEMORY:", PIN_MEMORY)
print("NUM_WORKERS:", NUM_WORKERS)

PIN_MEMORY: True
NUM_WORKERS: 0


In [13]:
gpu_count = torch.cuda.device_count()
print("GPU count:", gpu_count)
for i in range(gpu_count):
    print(f"  GPU {i}:", torch.cuda.get_device_name(i))

GPU count: 3
  GPU 0: Quadro RTX 4000
  GPU 1: Quadro P4000
  GPU 2: Quadro P4000


In [14]:
FORCE_USE_ALL_GPUS = False
USE_DATAPARALLEL = (device.type == "cuda" and gpu_count > 1)
print("USE_DATAPARALLEL:", USE_DATAPARALLEL)

USE_DATAPARALLEL: True


In [15]:
def choose_dp_device_ids(min_ratio=0.75):
    if gpu_count < 2:
        return None
    props = [torch.cuda.get_device_properties(i) for i in range(gpu_count)]
    mem0 = props[0].total_memory
    sm0  = props[0].multi_processor_count
    bad = []
    for i in range(1, gpu_count):
        mem_ratio = props[i].total_memory / mem0
        sm_ratio  = props[i].multi_processor_count / sm0
        ratio = min(mem_ratio, sm_ratio)
        if ratio < min_ratio:
            bad.append((i, mem_ratio, sm_ratio))
    if bad:
        print("DataParallel imbalance detectado. Usaré SOLO GPU0.")
        for (i, mr, sr) in bad:
            print(f" - GPU{i}: mem_ratio={mr:.2f}, sm_ratio={sr:.2f} (< {min_ratio})")
        return [0]
    return list(range(gpu_count))

In [16]:
DP_DEVICE_IDS = None

In [17]:
if USE_DATAPARALLEL:
    DP_DEVICE_IDS = list(range(gpu_count)) if FORCE_USE_ALL_GPUS else choose_dp_device_ids(0.75)
    if DP_DEVICE_IDS == [0]:
        USE_DATAPARALLEL = False
        DP_DEVICE_IDS = None

DataParallel imbalance detectado. Usaré SOLO GPU0.
 - GPU1: mem_ratio=1.00, sm_ratio=0.39 (< 0.75)
 - GPU2: mem_ratio=1.00, sm_ratio=0.39 (< 0.75)


In [18]:
print("USE_DATAPARALLEL:", USE_DATAPARALLEL)
print("DP_DEVICE_IDS   :", DP_DEVICE_IDS)

USE_DATAPARALLEL: False
DP_DEVICE_IDS   : None


In [19]:
def autocast_ctx():
    if device.type != "cuda":
        return nullcontext()
    return torch.amp.autocast(device_type="cuda", dtype=torch.float16)

In [20]:
def get_scaler():
    if device.type != "cuda":
        return None
    return torch.amp.GradScaler(device="cuda")

# Run state

In [22]:
def load_state():
    if STATE_PATH.exists():
        return json.loads(STATE_PATH.read_text(encoding="utf-8"))
    return {}

In [23]:
def save_state(st: dict):
    STATE_PATH.parent.mkdir(parents=True, exist_ok=True)
    STATE_PATH.write_text(json.dumps(st, indent=2), encoding="utf-8")

In [24]:
def mark_done(key: str, value=True):
    st = load_state()
    st[key] = value
    save_state(st)

In [25]:
def is_done(key: str):
    return bool(load_state().get(key, False))

In [26]:
def mark_pct_done(model_key: str, pct: int, value=True):
    st = load_state()
    if model_key not in st:
        st[model_key] = {}
    st[model_key][str(pct)] = value
    save_state(st)

In [27]:
def is_pct_done(model_key: str, pct: int):
    st = load_state()
    return bool(st.get(model_key, {}).get(str(pct), False))

# Descarga de Dataset Cityscapes

In [29]:
def download_cityscapes_with_kagglehub():
    path = kagglehub.dataset_download("electraawais/cityscape-dataset")
    print("Path to dataset files:", path)
    return Path(path)

In [30]:
DATASET_ROOT = download_cityscapes_with_kagglehub()

Path to dataset files: C:\Users\ALLAN TURING\.cache\kagglehub\datasets\electraawais\cityscape-dataset\versions\2


In [31]:
def resolve_cityscapes_paths(root: Path):
    known_img = root / "Cityscape Dataset" / "leftImg8bit"
    known_gt  = root / "Fine Annotations" / "gtFine"
    if known_img.is_dir() and known_gt.is_dir():
        return known_img, known_gt

    left_candidates = []
    gt_candidates   = []
    for p in root.rglob("leftImg8bit"):
        if (p / "train").is_dir() and (p / "val").is_dir():
            left_candidates.append(p)
    for p in root.rglob("gtFine"):
        if (p / "train").is_dir() and (p / "val").is_dir():
            gt_candidates.append(p)

    if not left_candidates or not gt_candidates:
        raise FileNotFoundError("No encontré leftImg8bit/gtFine con train/val en el dataset descargado.")
    return left_candidates[0], gt_candidates[0]

In [32]:
IMG_ROOT, GT_ROOT = resolve_cityscapes_paths(DATASET_ROOT)

In [33]:
print("IMG_ROOT:", IMG_ROOT)
print("GT_ROOT :", GT_ROOT)

IMG_ROOT: C:\Users\ALLAN TURING\.cache\kagglehub\datasets\electraawais\cityscape-dataset\versions\2\Cityscape Dataset\leftImg8bit
GT_ROOT : C:\Users\ALLAN TURING\.cache\kagglehub\datasets\electraawais\cityscape-dataset\versions\2\Fine Annotations\gtFine


In [34]:
for sp in ["train", "val", "test"]:
    assert (IMG_ROOT / sp).is_dir(), f"Falta {sp} en leftImg8bit"
    assert (GT_ROOT / sp).is_dir(), f"Falta {sp} en gtFine"

# Construcción de pares (imagen - máscara) por split

In [36]:
def collect_pairs(split: str):
    img_glob = str(IMG_ROOT / split / "*" / "*_leftImg8bit.png")
    imgs = sorted(glob.glob(img_glob))
    pairs, missing = [], 0
    for ip in imgs:
        ip = Path(ip)
        city = ip.parent.name
        stem = ip.name.replace("_leftImg8bit.png", "")
        mp = GT_ROOT / split / city / f"{stem}_gtFine_labelIds.png"
        if mp.exists():
            pairs.append((str(ip), str(mp)))
        else:
            missing += 1
    print(f"[{split}] imgs={len(imgs)} paired={len(pairs)} missing_masks={missing}")
    return pairs

In [37]:
train_pairs_full = collect_pairs("train")
val_pairs  = collect_pairs("val")
test_pairs = collect_pairs("test")

[train] imgs=2975 paired=2975 missing_masks=0
[val] imgs=500 paired=500 missing_masks=0
[test] imgs=1525 paired=1525 missing_masks=0


In [38]:
assert len(train_pairs_full) > 0 and len(val_pairs) > 0, "No hay pares suficientes."

### Guardar splits

In [40]:
def write_split(pairs, filename):
    outp = SPLITS_DIR / filename
    with open(outp, "w", encoding="utf-8") as f:
        for img, mask in pairs:
            f.write(f"{img}\t{mask}\n")
    return outp

In [41]:
print("Splits guardados:")
print(" -", write_split(train_pairs_full, "train.txt"))
print(" -", write_split(val_pairs, "val.txt"))
print(" -", write_split(test_pairs, "test.txt"))
mark_done("splits_written", True)

Splits guardados:
 - C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\splits\train.txt
 - C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\splits\val.txt
 - C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\splits\test.txt


# Trainlds 19 clases

In [43]:
ID_TO_TRAINID = np.ones(256, dtype=np.uint8) * IGNORE_INDEX
mapping = {
    7: 0,   # road
    8: 1,   # sidewalk
    11: 2,  # building
    12: 3,  # wall
    13: 4,  # fence
    17: 5,  # pole
    19: 6,  # traffic light
    20: 7,  # traffic sign
    21: 8,  # vegetation
    22: 9,  # terrain
    23: 10, # sky
    24: 11, # person
    25: 12, # rider
    26: 13, # car
    27: 14, # truck
    28: 15, # bus
    31: 16, # train
    32: 17, # motorcycle
    33: 18  # bicycle
}

In [44]:
for k, v in mapping.items():
    ID_TO_TRAINID[k] = v

In [45]:
id2label = {
    0:"road",1:"sidewalk",2:"building",3:"wall",4:"fence",5:"pole",
    6:"traffic_light",7:"traffic_sign",8:"vegetation",9:"terrain",
    10:"sky",11:"person",12:"rider",13:"car",14:"truck",15:"bus",
    16:"train",17:"motorcycle",18:"bicycle"
}
label2id = {v:k for k,v in id2label.items()}

In [46]:
SELECTED = {"road":0, "person":11, "car":13, "building":2, "vegetation":8}

In [47]:
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32).view(3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32).view(3,1,1)

# Augment

In [49]:
def pad_to_min_size(img_np, mask_np, min_h, min_w):
    h, w = img_np.shape[0], img_np.shape[1]
    pad_h = max(0, min_h - h)
    pad_w = max(0, min_w - w)
    if pad_h > 0 or pad_w > 0:
        img_np = np.pad(img_np, ((0, pad_h), (0, pad_w), (0, 0)), mode="reflect")
        mask_np = np.pad(mask_np, ((0, pad_h), (0, pad_w)), mode="constant", constant_values=IGNORE_INDEX)
    return img_np, mask_np

In [50]:
def random_crop(img_np, mask_np, crop_h, crop_w):
    h, w = img_np.shape[0], img_np.shape[1]
    y0 = random.randint(0, h - crop_h)
    x0 = random.randint(0, w - crop_w)
    return img_np[y0:y0+crop_h, x0:x0+crop_w, :], mask_np[y0:y0+crop_h, x0:x0+crop_w]

In [51]:
def color_jitter_np(img_np, brightness=0.2, contrast=0.2, saturation=0.2):
    b = 1.0 + random.uniform(-brightness, brightness)
    c = 1.0 + random.uniform(-contrast, contrast)
    s = 1.0 + random.uniform(-saturation, saturation)
    img = np.clip(img_np * b, 0, 1)
    mean = img.mean(axis=(0,1), keepdims=True)
    img = np.clip((img - mean) * c + mean, 0, 1)
    gray = (img[...,0]*0.299 + img[...,1]*0.587 + img[...,2]*0.114)[...,None]
    img = np.clip((img - gray) * s + gray, 0, 1)
    return img

In [52]:
def pil_color_jitter(img_pil, b=0.2, c=0.2, s=0.2):
    img_pil = ImageEnhance.Brightness(img_pil).enhance(1.0 + random.uniform(-b, b))
    img_pil = ImageEnhance.Contrast(img_pil).enhance(1.0 + random.uniform(-c, c))
    img_pil = ImageEnhance.Color(img_pil).enhance(1.0 + random.uniform(-s, s))
    return img_pil

# Corruptions

In [54]:
def apply_motion_blur(img_pil: Image.Image, k: int = 9):
    k = int(k)
    if k < 3:
        k = 3
    if k % 2 == 0:
        k += 1
    k = min(k, 15)
    
    try:
        import cv2
        arr = np.array(img_pil.convert("RGB"), dtype=np.uint8)
        kernel = np.zeros((k, k), dtype=np.float32)
        kernel[k // 2, :] = 1.0 / k
        out = cv2.filter2D(arr, ddepth=-1, kernel=kernel)
        return Image.fromarray(out, mode="RGB")
    except Exception:
        radius = max(1, k // 4)
        return img_pil.filter(ImageFilter.GaussianBlur(radius=float(radius)))

In [55]:
def apply_gaussian_noise(img_np_uint8, sigma: float = 15.0):
    noise = np.random.normal(0, float(sigma), img_np_uint8.shape).astype(np.float32)
    out = np.clip(img_np_uint8.astype(np.float32) + noise, 0, 255).astype(np.uint8)
    return out

In [56]:
def apply_jpeg_compression(img_pil: Image.Image, quality: int = 30):
    buf = BytesIO()
    img_pil.convert("RGB").save(buf, format="JPEG", quality=int(quality))
    buf.seek(0)
    out = Image.open(buf).convert("RGB").copy()  # copy() evita problemas en workers
    buf.close()
    return out

In [57]:
def apply_corruption(img_pil: Image.Image, corruption: str, severity: int = 1):
    severity = int(np.clip(int(severity), 1, 5))

    try:
        if corruption == "gaussian_blur":
            radius = [0.5, 1.0, 1.5, 2.0, 3.0][severity-1]
            return img_pil.filter(ImageFilter.GaussianBlur(radius=float(radius)))

        if corruption == "motion_blur":
            k = [5, 7, 9, 11, 15][severity-1]
            return apply_motion_blur(img_pil, k=k)

        if corruption == "gaussian_noise":
            sigma = [5, 10, 15, 25, 35][severity-1]
            arr = np.array(img_pil.convert("RGB"), dtype=np.uint8)
            arr = apply_gaussian_noise(arr, sigma=sigma)
            return Image.fromarray(arr, mode="RGB")

        if corruption == "brightness":
            factor = [0.7, 0.6, 0.5, 0.4, 0.3][severity-1]
            return ImageEnhance.Brightness(img_pil).enhance(float(factor))

        if corruption == "contrast":
            factor = [0.8, 0.7, 0.6, 0.5, 0.4][severity-1]
            return ImageEnhance.Contrast(img_pil).enhance(float(factor))

        if corruption == "jpeg":
            quality = [60, 45, 30, 20, 10][severity-1]
            return apply_jpeg_compression(img_pil, quality=int(quality))

        return img_pil

    except Exception as e:
        if not hasattr(apply_corruption, "_warned"):
            print(f"[WARN] Corruption '{corruption}' failed (severity={severity}): {e}. Using clean image.")
            apply_corruption._warned = True
        return img_pil

In [58]:
### Test de corruptions ok
img = Image.new("RGB", (512, 1024), color=(128,128,128))
for corr in ["gaussian_blur","motion_blur","gaussian_noise","brightness","contrast","jpeg"]:
    for s in range(1,6):
        _ = apply_corruption(img, corr, s)
    print(corr)
print("Corruptions OK")

gaussian_blur
motion_blur
gaussian_noise
brightness
contrast
jpeg
Corruptions OK


# Datasets (CNN y SegFormer)

In [60]:
class CityscapesDatasetCNN(Dataset):
    def __init__(self, pairs, resize=RESIZE, crop=CROP, train=False, corruption=None, severity=1):
        self.pairs = pairs
        self.resize = resize
        self.crop = crop
        self.train = train
        self.corruption = corruption
        self.severity = severity

    def __len__(self): return len(self.pairs)

    def __getitem__(self, i):
        img_path, mask_path = self.pairs[i]
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        img  = img.resize((self.resize[1], self.resize[0]), resample=Image.BILINEAR)
        mask = mask.resize((self.resize[1], self.resize[0]), resample=Image.NEAREST)

        if (not self.train) and (self.corruption is not None):
            img = apply_corruption(img, self.corruption, self.severity)

        img_np  = np.array(img, dtype=np.float32) / 255.0
        mask_np = ID_TO_TRAINID[np.array(mask, dtype=np.uint8)].astype(np.int64)

        if self.train:
            if random.random() < 0.5:
                img_np  = np.ascontiguousarray(img_np[:, ::-1, :])
                mask_np = np.ascontiguousarray(mask_np[:, ::-1])
            img_np = color_jitter_np(img_np, 0.2, 0.2, 0.2)

            ch, cw = self.crop
            img_np, mask_np = pad_to_min_size(img_np, mask_np, ch, cw)
            img_np, mask_np = random_crop(img_np, mask_np, ch, cw)

        img_t = torch.from_numpy(img_np).permute(2,0,1).contiguous()
        img_t = (img_t - IMAGENET_MEAN) / IMAGENET_STD
        mask_t = torch.from_numpy(mask_np.astype(np.int64))
        return img_t, mask_t

In [61]:
processor = SegformerImageProcessor.from_pretrained(
    MODEL_ID,
    do_rescale=True,
    do_normalize=True
)

C:\Users\ALLAN TURING\anaconda3\Lib\site-packages\transformers\image_processing_base.py:417: UserWarning: The following named arguments are not valid for `SegformerImageProcessor.__init__` and were ignored: 'feature_extractor_type', 'reduce_labels'
  image_processor = cls(**image_processor_dict)


In [62]:
class CityscapesDatasetSF(Dataset):
    def __init__(self, pairs, resize=RESIZE, crop=CROP, train=False, corruption=None, severity=1):
        self.pairs = pairs
        self.resize = resize
        self.crop = crop
        self.train = train
        self.corruption = corruption
        self.severity = severity

    def __len__(self): return len(self.pairs)

    def __getitem__(self, i):
        img_path, mask_path = self.pairs[i]
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)

        img  = img.resize((self.resize[1], self.resize[0]), resample=Image.BILINEAR)
        mask = mask.resize((self.resize[1], self.resize[0]), resample=Image.NEAREST)

        if (not self.train) and (self.corruption is not None):
            img = apply_corruption(img, self.corruption, self.severity)

        mask_np = ID_TO_TRAINID[np.array(mask, dtype=np.uint8)].astype(np.int64)

        if self.train:
            if random.random() < 0.5:
                img_np  = np.array(img, dtype=np.uint8)
                img_np  = np.ascontiguousarray(img_np[:, ::-1, :])
                mask_np = np.ascontiguousarray(mask_np[:, ::-1])
                img = Image.fromarray(img_np)
            img = pil_color_jitter(img, b=0.2, c=0.2, s=0.2)

            img_np = np.array(img, dtype=np.uint8)
            ch, cw = self.crop
            img_np, mask_np = pad_to_min_size(img_np, mask_np, ch, cw)
            img_np, mask_np = random_crop(img_np, mask_np, ch, cw)
            img = Image.fromarray(img_np)

        enc = processor(images=img, return_tensors="pt")
        pixel_values = enc["pixel_values"].squeeze(0)
        labels = torch.from_numpy(mask_np.astype(np.int64))
        return pixel_values, labels

# Métricas

In [64]:
@torch.no_grad()
def update_confusion(conf, preds, labels, num_classes=NUM_CLASSES, ignore_index=IGNORE_INDEX):
    valid = labels != ignore_index
    gt = labels[valid].view(-1)
    pd = preds[valid].view(-1)
    k = (gt * num_classes + pd).to(torch.int64)
    conf += torch.bincount(k, minlength=num_classes*num_classes).reshape(num_classes, num_classes)
    return conf

In [65]:
def metrics_from_conf(conf):
    tp = conf.diag().float()
    fp = conf.sum(0).float() - tp
    fn = conf.sum(1).float() - tp
    iou = tp / (tp + fp + fn + 1e-6)
    miou = iou.mean().item()
    return miou, iou.detach().cpu().numpy()

In [66]:
def pixel_acc_from_conf(conf):
    correct = conf.diag().sum().item()
    total = conf.sum().item()
    return correct / (total + 1e-6)

In [67]:
def selected_class_iou(iou_arr):
    return {k: float(iou_arr[idx]) for k, idx in SELECTED.items()}

# Builders

In [69]:
def build_cnn_model(num_classes=NUM_CLASSES):
    try:
        backbone_w = ResNet50_Weights.DEFAULT
    except Exception:
        backbone_w = None

    try:
        model = deeplabv3_resnet50(weights=None, weights_backbone=backbone_w, num_classes=num_classes)
    except Exception:
        model = deeplabv3_resnet50(weights=None, weights_backbone=None, num_classes=num_classes)

    if USE_DATAPARALLEL and DP_DEVICE_IDS is not None:
        model = nn.DataParallel(model, device_ids=DP_DEVICE_IDS)
    return model.to(device)

In [70]:
class SegFormerWrapper(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base = base_model

    def forward(self, pixel_values):
        out = self.base(pixel_values=pixel_values)
        return {"logits": out.logits}

In [71]:
def build_segformer_model(model_id=MODEL_ID):
    base = SegformerForSemanticSegmentation.from_pretrained(
        model_id,
        num_labels=NUM_CLASSES,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )
    model = SegFormerWrapper(base)
    if USE_DATAPARALLEL and DP_DEVICE_IDS is not None:
        model = nn.DataParallel(model, device_ids=DP_DEVICE_IDS)
    return model.to(device)

In [72]:
def save_pth_weights(model, path: Path):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    state = model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict()
    torch.save(state, str(path))
    return path

In [73]:
def load_pth_weights(model, path: Path, strict=True):
    path = Path(path)
    state = torch.load(str(path), map_location="cpu", weights_only=True)
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(state, strict=strict)
    else:
        model.load_state_dict(state, strict=strict)
    return model

# Evaluates

In [75]:
@torch.no_grad()
def evaluate_cnn(model, loader):
    model.eval()
    conf = torch.zeros((NUM_CLASSES, NUM_CLASSES), dtype=torch.int64, device=device)
    for imgs, masks in loader:
        imgs  = imgs.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)
        out = model(imgs)["out"]
        preds = out.argmax(1)
        conf = update_confusion(conf, preds, masks)
    miou, iou_arr = metrics_from_conf(conf)
    acc = pixel_acc_from_conf(conf)
    return miou, acc, selected_class_iou(iou_arr)

In [76]:
@torch.no_grad()
def evaluate_sf(model_sf, loader):
    model_sf.eval()
    conf = torch.zeros((NUM_CLASSES, NUM_CLASSES), dtype=torch.int64, device=device)
    for px, labels in loader:
        px     = px.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        logits = model_sf(pixel_values=px)["logits"]
        logits = F.interpolate(logits, size=labels.shape[-2:], mode="bilinear", align_corners=False)
        preds  = logits.argmax(1)
        conf = update_confusion(conf, preds, labels)
    miou, iou_arr = metrics_from_conf(conf)
    acc = pixel_acc_from_conf(conf)
    return miou, acc, selected_class_iou(iou_arr)

# Train

In [78]:
def train_one_epoch_cnn(model, loader, criterion, optimizer, scaler, epoch, epochs):
    model.train()
    running = 0.0
    pbar = tqdm(loader, desc=f"CNN Epoch {epoch}/{epochs}", leave=True)
    t0 = time.time()
    for step, (imgs, masks) in enumerate(pbar, start=1):
        imgs  = imgs.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast_ctx():
            out = model(imgs)["out"]
            loss = criterion(out, masks)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        running += float(loss.item())
        avg_loss = running / step
        ms_step = ((time.time() - t0) / step) * 1000
        pbar.set_postfix(loss=f"{avg_loss:.4f}", ms_step=f"{ms_step:.0f}", lr=f"{optimizer.param_groups[0]['lr']:.2e}")
    return running / max(len(loader), 1)

In [79]:
def train_one_epoch_sf(model_sf, loader, optimizer, scheduler, scaler, epoch, epochs):
    model_sf.train()
    running = 0.0
    pbar = tqdm(loader, desc=f"SF Epoch {epoch}/{epochs}", leave=True)
    t0 = time.time()

    for step, (px, labels) in enumerate(pbar, start=1):
        px = px.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast_ctx():
            out = model_sf(pixel_values=px)
            logits = out["logits"]
            logits = F.interpolate(
                logits, size=labels.shape[-2:], mode="bilinear", align_corners=False
            )
            loss = F.cross_entropy(logits, labels, ignore_index=IGNORE_INDEX)

        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        scheduler.step()

        running += float(loss.item())
        avg_loss = running / step
        ms_step = ((time.time() - t0) / step) * 1000
        pbar.set_postfix(loss=f"{avg_loss:.4f}", ms_step=f"{ms_step:.0f}",
                         lr=f"{optimizer.param_groups[0]['lr']:.2e}")

    return running / max(len(loader), 1)

# Dataloaders

In [81]:
def make_train_subset(pairs, pct, seed=SEED):
    pct = int(pct)
    assert pct in [10, 25, 50, 100]
    if pct == 100:
        return pairs
    n = len(pairs)
    m = max(1, int(n * (pct / 100.0)))
    rng = np.random.RandomState(seed + pct)
    idx = rng.permutation(n)[:m]
    return [pairs[i] for i in idx]

In [82]:
def make_loader_cnn(pairs, train: bool, corruption=None, severity=1, batch_size=None):
    bs = CNN_BATCH if batch_size is None else batch_size
    ds = CityscapesDatasetCNN(pairs, resize=RESIZE, crop=CROP, train=train, corruption=corruption, severity=severity)
    return DataLoader(ds, batch_size=bs, shuffle=train, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, drop_last=train)

In [83]:
def make_loader_sf(pairs, train: bool, corruption=None, severity=1, batch_size=None):
    bs = SF_BATCH if batch_size is None else batch_size
    ds = CityscapesDatasetSF(pairs, resize=RESIZE, crop=CROP, train=train, corruption=corruption, severity=severity)
    return DataLoader(ds, batch_size=bs, shuffle=train, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, drop_last=train)

In [84]:
def upsert_row_csv(path: Path, key_cols, row_dict):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    if path.exists():
        df = pd.read_csv(path)
    else:
        df = pd.DataFrame(columns=list(row_dict.keys()))
    for c in row_dict.keys():
        if c not in df.columns:
            df[c] = np.nan
    if len(df) == 0:
        df = pd.DataFrame([row_dict])
    else:
        mask = np.ones(len(df), dtype=bool)
        for kc in key_cols:
            mask &= (df[kc].astype(str) == str(row_dict[kc]))
        if mask.any():
            df.loc[mask, list(row_dict.keys())] = list(row_dict.values())
        else:
            df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)
    df.to_csv(path, index=False)

# Train CNN

In [86]:
def train_cnn_pct(pct: int):
    ckpt_path = CKPT_DIR / f"best_cnn_pct{pct}.pth"
    hist_path = LOG_DIR  / f"history_cnn_pct{pct}.csv"

    if ckpt_path.exists() and is_pct_done("cnn_done", pct):
        print(f"[SKIP] CNN pct={pct} ya entrenado. CKPT: {ckpt_path}")
        model = build_cnn_model()
        load_pth_weights(model, ckpt_path)
        return model, ckpt_path, hist_path

    train_pairs = make_train_subset(train_pairs_full, pct, seed=SEED)
    train_loader = make_loader_cnn(train_pairs, train=True)
    val_loader   = make_loader_cnn(val_pairs, train=False)

    # Smoke test (detect hang early)
    print("[DEBUG] CNN loader smoke test...")
    b = next(iter(train_loader))
    print("[DEBUG] CNN batch OK:", b[0].shape, b[1].shape)

    model = build_cnn_model()
    criterion = nn.CrossEntropyLoss(ignore_index=IGNORE_INDEX)
    optimizer = optim.AdamW(model.parameters(), lr=CNN_LR, weight_decay=CNN_WD)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CNN_EPOCHS)
    scaler = get_scaler()

    best_miou = -1.0
    bad = 0
    rows = []

    for e in range(1, CNN_EPOCHS + 1):
        loss = train_one_epoch_cnn(model, train_loader, criterion, optimizer, scaler, e, CNN_EPOCHS)
        miou, acc, cls = evaluate_cnn(model, val_loader)

        improved = False
        if miou > best_miou:
            best_miou = miou
            bad = 0
            save_pth_weights(model, ckpt_path)
            improved = True
        else:
            bad += 1

        scheduler.step()

        print(f"[CNN pct={pct}] Epoch {e}/{CNN_EPOCHS} | loss={loss:.4f} | val mIoU={miou:.4f} | acc={acc:.4f} {'(best)' if improved else ''}")

        rows.append({
            "pct": pct, "epoch": e,
            "train_loss": loss, "val_miou": miou, "val_acc": acc,
            "lr": optimizer.param_groups[0]["lr"],
            "iou_road": cls["road"], "iou_person": cls["person"], "iou_car": cls["car"],
            "iou_building": cls["building"], "iou_vegetation": cls["vegetation"]
        })
        pd.DataFrame(rows).to_csv(hist_path, index=False)

        if bad >= CNN_PATIENCE:
            print(f"[CNN pct={pct}] Early stopping.")
            break

    mark_pct_done("cnn_done", pct, True)
    load_pth_weights(model, ckpt_path)
    return model, ckpt_path, hist_path

In [87]:
def stage_train_cnn_all_pcts():
    out_summary = TABLE_DIR / "cnn_data_efficiency_summary.csv"

    if is_done("cnn_data_eff_done"):
        print("[SKIP] CNN data-efficiency ya completado, no re-evaluo pcts.")
        return load_best_cnn_100()
        
    best_cnn_100 = None

    for pct in DATA_EFF_PCTS:
        print(f"CNN DATA EFFICIENCY: pct={pct} | train_samples={len(make_train_subset(train_pairs_full, pct))}")
        cnn_model, cnn_ckpt, cnn_hist = train_cnn_pct(pct)

        # Store best val (evaluate now)
        val_loader = make_loader_cnn(val_pairs, train=False)
        best_val_miou, best_val_acc, _ = evaluate_cnn(cnn_model, val_loader)

        upsert_row_csv(out_summary, key_cols=["pct"], row_dict={
            "pct": pct,
            "best_val_miou": best_val_miou,
            "best_val_acc": best_val_acc,
            "cnn_ckpt": str(cnn_ckpt),
            "cnn_hist": str(cnn_hist),
        })
        print("Saved/Updated:", out_summary)

        if pct == 100:
            best_cnn_100 = cnn_model

    mark_done("cnn_data_eff_done", True)
    return best_cnn_100

# Train SegFormer

In [89]:
def train_sf_pct(pct: int):
    ckpt_path = CKPT_DIR / f"best_sf_pct{pct}.pth"
    hist_path = LOG_DIR  / f"history_sf_pct{pct}.csv"

    if ckpt_path.exists() and is_pct_done("sf_done", pct):
        print(f"[SKIP] SF pct={pct} ya entrenado. CKPT: {ckpt_path}")
        model = build_segformer_model()
        load_pth_weights(model, ckpt_path, strict=False)
        return model, ckpt_path, hist_path

    train_pairs = make_train_subset(train_pairs_full, pct, seed=SEED)
    train_loader = make_loader_sf(train_pairs, train=True)
    val_loader   = make_loader_sf(val_pairs, train=False)

    # Smoke test (detect hang early)
    print("[DEBUG] SF loader smoke test...")
    b = next(iter(train_loader))
    print("[DEBUG] SF batch OK:", b[0].shape, b[1].shape)

    model = build_segformer_model()

    optimizer = optim.AdamW(model.parameters(), lr=SF_LR, weight_decay=SF_WD)
    total_steps  = SF_EPOCHS * len(train_loader)
    warmup_steps = int(0.10 * total_steps)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    scaler = get_scaler()

    best_miou = -1.0
    bad = 0
    rows = []

    for e in range(1, SF_EPOCHS + 1):
        loss = train_one_epoch_sf(model, train_loader, optimizer, scheduler, scaler, e, SF_EPOCHS)
        miou, acc, cls = evaluate_sf(model, val_loader)

        improved = False
        if miou > best_miou:
            best_miou = miou
            bad = 0
            save_pth_weights(model, ckpt_path)
            improved = True
        else:
            bad += 1

        print(f"[SF pct={pct}] Epoch {e}/{SF_EPOCHS} | loss={loss:.4f} | val mIoU={miou:.4f} | acc={acc:.4f} {'(best)' if improved else ''}")

        rows.append({
            "pct": pct, "epoch": e,
            "train_loss": loss, "val_miou": miou, "val_acc": acc,
            "lr": optimizer.param_groups[0]["lr"],
            "iou_road": cls["road"], "iou_person": cls["person"], "iou_car": cls["car"],
            "iou_building": cls["building"], "iou_vegetation": cls["vegetation"]
        })
        pd.DataFrame(rows).to_csv(hist_path, index=False)

        if bad >= SF_PATIENCE:
            print(f"[SF pct={pct}] Early stopping.")
            break

    mark_pct_done("sf_done", pct, True)
    load_pth_weights(model, ckpt_path, strict=False)
    return model, ckpt_path, hist_path

In [90]:
def stage_train_sf_all_pcts():
    out_summary = TABLE_DIR / "sf_data_efficiency_summary.csv"
    best_sf_100 = None

    for pct in DATA_EFF_PCTS:
        print(f"SF DATA EFFICIENCY: pct={pct} | train_samples={len(make_train_subset(train_pairs_full, pct))}")
        sf_model, sf_ckpt, sf_hist = train_sf_pct(pct)

        # Store best val (evaluate now)
        val_loader = make_loader_sf(val_pairs, train=False)
        best_val_miou, best_val_acc, _ = evaluate_sf(sf_model, val_loader)

        upsert_row_csv(out_summary, key_cols=["pct"], row_dict={
            "pct": pct,
            "best_val_miou": best_val_miou,
            "best_val_acc": best_val_acc,
            "sf_ckpt": str(sf_ckpt),
            "sf_hist": str(sf_hist),
        })
        print("Saved/Updated:", out_summary)

        if pct == 100:
            best_sf_100 = sf_model

    mark_done("sf_data_eff_done", True)
    return best_sf_100

# Plots

In [92]:
def stage_plot_curve_cnn():
    csv_path = TABLE_DIR / "cnn_data_efficiency_summary.csv"
    if not csv_path.exists():
        print("[WARN] No existe cnn_data_efficiency_summary.csv. Entrena CNN primero.")
        return
    df = pd.read_csv(csv_path).sort_values("pct")
    fig_path = FIG_DIR / "data_efficiency_curve_cnn.png"
    plt.figure(figsize=(7,4))
    plt.plot(df["pct"], df["best_val_miou"], marker="o", label="CNN (DeepLabV3)")
    plt.xlabel("% training data"); plt.ylabel("Best Val mIoU")
    plt.title("Data efficiency (CNN): mIoU vs % training data")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig(fig_path, dpi=200); plt.close()
    print("Saved:", fig_path)
    mark_done("curve_cnn_done", True)

In [93]:
def stage_plot_curve_sf():
    csv_path = TABLE_DIR / "sf_data_efficiency_summary.csv"
    if not csv_path.exists():
        print("[WARN] No existe sf_data_efficiency_summary.csv. Entrena SF primero.")
        return
    df = pd.read_csv(csv_path).sort_values("pct")
    fig_path = FIG_DIR / "data_efficiency_curve_sf.png"
    plt.figure(figsize=(7,4))
    plt.plot(df["pct"], df["best_val_miou"], marker="o", label="SegFormer")
    plt.xlabel("% training data"); plt.ylabel("Best Val mIoU")
    plt.title("Data efficiency (SegFormer): mIoU vs % training data")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig(fig_path, dpi=200); plt.close()
    print("Saved:", fig_path)
    mark_done("curve_sf_done", True)

In [94]:
def stage_plot_curve_compare():
    cnn_csv = TABLE_DIR / "cnn_data_efficiency_summary.csv"
    sf_csv  = TABLE_DIR / "sf_data_efficiency_summary.csv"
    if (not cnn_csv.exists()) or (not sf_csv.exists()):
        print("[WARN] Necesitas ambos CSV (CNN y SF) para la curva comparativa.")
        return
    d1 = pd.read_csv(cnn_csv).sort_values("pct")[["pct","best_val_miou"]].rename(columns={"best_val_miou":"cnn_miou"})
    d2 = pd.read_csv(sf_csv).sort_values("pct")[["pct","best_val_miou"]].rename(columns={"best_val_miou":"sf_miou"})
    df = d1.merge(d2, on="pct")
    fig_path = FIG_DIR / "data_efficiency_curve_compare.png"
    plt.figure(figsize=(7,4))
    plt.plot(df["pct"], df["cnn_miou"], marker="o", label="CNN (DeepLabV3)")
    plt.plot(df["pct"], df["sf_miou"],  marker="o", label="SegFormer")
    plt.xlabel("% training data"); plt.ylabel("Best Val mIoU")
    plt.title("Data efficiency: CNN vs SegFormer (Cityscapes)")
    plt.grid(True); plt.legend(); plt.tight_layout()
    plt.savefig(fig_path, dpi=200); plt.close()
    print("Saved:", fig_path)
    mark_done("curve_compare_done", True)

# Eficiencia (latencia + VRAM + params)

In [96]:
def count_params_m(model):
    return sum(p.numel() for p in model.parameters()) / 1e6

In [97]:
@torch.no_grad()
def measure_inference(model, loader, forward_fn, n_warmup=EFF_WARMUP_STEPS, n_steps=EFF_MEASURE_STEPS):
    model.eval()
    if device.type == "cuda":
        for gi in range(torch.cuda.device_count()):
            torch.cuda.reset_peak_memory_stats(gi)

    it = iter(loader)
    for _ in range(n_warmup):
        try:
            batch = next(it)
        except StopIteration:
            it = iter(loader)
            batch = next(it)
        _ = forward_fn(batch)

    if device.type == "cuda":
        torch.cuda.synchronize()

    t0 = time.time()
    n_imgs = 0
    for _ in range(n_steps):
        try:
            batch = next(it)
        except StopIteration:
            it = iter(loader)
            batch = next(it)
        _ = forward_fn(batch)
        n_imgs += batch[0].shape[0]

    if device.type == "cuda":
        torch.cuda.synchronize()

    dt = time.time() - t0
    ms_per_img = (dt / max(n_imgs, 1)) * 1000

    vram = {}
    if device.type == "cuda":
        for gi in range(torch.cuda.device_count()):
            vram[gi] = torch.cuda.max_memory_allocated(gi) / (1024**3)
    return ms_per_img, vram

In [98]:
def load_best_cnn_100():
    ckpt = CKPT_DIR / "best_cnn_pct100.pth"
    if not ckpt.exists():
        raise FileNotFoundError("No existe best_cnn_pct100.pth. Entrena CNN (pct100) primero.")
    model = build_cnn_model()
    load_pth_weights(model, ckpt)
    model.to(device)
    model.eval()
    return model

In [99]:
def load_best_sf_100():
    ckpt = CKPT_DIR / "best_sf_pct100.pth"
    if not ckpt.exists():
        raise FileNotFoundError("No existe best_sf_pct100.pth. Entrena SF (pct100) primero.")
    model = build_segformer_model()
    load_pth_weights(model, ckpt, strict=False)
    return model

In [100]:
def stage_efficiency_cnn():
    cnn = load_best_cnn_100()
    val_loader = make_loader_cnn(val_pairs, train=False)

    def fwd(batch):
        imgs, _ = batch
        imgs = imgs.to(device, non_blocking=True)
        out = cnn(imgs)["out"]
        _ = out.argmax(1)
        return out

    ms, vram = measure_inference(cnn, val_loader, fwd)
    out_path = TABLE_DIR / "efficiency_cnn.csv"
    pd.DataFrame([{
        "model":"CNN (DeepLabV3)",
        "latency_ms_per_img": ms,
        "params_m": count_params_m(cnn),
        "vram_gb_gpu0": vram.get(0, None),
        "vram_gb_gpu1": vram.get(1, None),
        "vram_gb_gpu2": vram.get(2, None),
    }]).to_csv(out_path, index=False)
    print("Saved:", out_path)
    mark_done("eff_cnn_done", True)

In [101]:
def stage_efficiency_sf():
    sf = load_best_sf_100()
    val_loader = make_loader_sf(val_pairs, train=False)

    def fwd(batch):
        px, _ = batch
        px = px.to(device, non_blocking=True)
        out = sf(pixel_values=px)["logits"]
        _ = out.argmax(1)
        return out

    ms, vram = measure_inference(sf, val_loader, fwd)
    out_path = TABLE_DIR / "efficiency_sf.csv"
    pd.DataFrame([{
        "model":"SegFormer",
        "latency_ms_per_img": ms,
        "params_m": count_params_m(sf),
        "vram_gb_gpu0": vram.get(0, None),
        "vram_gb_gpu1": vram.get(1, None),
        "vram_gb_gpu2": vram.get(2, None),
    }]).to_csv(out_path, index=False)
    print("Saved:", out_path)
    mark_done("eff_sf_done", True)

In [102]:
def stage_efficiency_compare():
    cnn_csv = TABLE_DIR / "efficiency_cnn.csv"
    sf_csv  = TABLE_DIR / "efficiency_sf.csv"
    if (not cnn_csv.exists()) or (not sf_csv.exists()):
        print("[WARN] Necesitas efficiency_cnn.csv y efficiency_sf.csv para comparar.")
        return
    df = pd.concat([pd.read_csv(cnn_csv), pd.read_csv(sf_csv)], ignore_index=True)
    out_path = TABLE_DIR / "efficiency_compare.csv"
    df.to_csv(out_path, index=False)
    print("Saved:", out_path)
    mark_done("eff_compare_done", True)

# VISUALIZACIÓN: GT vs Pred + clean vs corrupted

In [104]:
CITYSCAPES_COLORS = np.array([
    [128,  64, 128],[244,  35, 232],[ 70,  70,  70],[102, 102, 156],[190, 153, 153],
    [153, 153, 153],[250, 170,  30],[220, 220,   0],[107, 142,  35],[152, 251, 152],
    [ 70, 130, 180],[220,  20,  60],[255,   0,   0],[  0,   0, 142],[  0,   0,  70],
    [  0,  60, 100],[  0,  80, 100],[  0,   0, 230],[119,  11,  32],
], dtype=np.uint8)

In [105]:
def colorize_trainid(mask_trainid):
    h, w = mask_trainid.shape
    out = np.zeros((h, w, 3), dtype=np.uint8)
    valid = (mask_trainid != IGNORE_INDEX)
    out[valid] = CITYSCAPES_COLORS[mask_trainid[valid]]
    return out

In [106]:
@torch.inference_mode()
def predict_cnn_single(img_pil, cnn_model):
    cnn_model.eval()
    img_r = img_pil.resize((RESIZE[1], RESIZE[0]), resample=Image.BILINEAR)
    img_np = np.array(img_r, dtype=np.float32) / 255.0

    x = torch.from_numpy(img_np).permute(2,0,1).unsqueeze(0).to(device)
    x = (x - IMAGENET_MEAN.to(device)) / IMAGENET_STD.to(device)

    out = cnn_model(x)
    logits = out["out"] if isinstance(out, dict) else out
    pred = logits.argmax(1).squeeze(0).cpu().numpy().astype(np.int64)
    return pred, img_r

In [107]:
@torch.no_grad()
def predict_sf_single(img_pil, sf_model):
    img_r = img_pil.resize((RESIZE[1], RESIZE[0]), resample=Image.BILINEAR)
    enc = processor(images=img_r, return_tensors="pt")
    px = enc["pixel_values"].to(device)
    logits = sf_model(pixel_values=px)["logits"]
    pred = logits.argmax(1).squeeze(0).detach().cpu().numpy().astype(np.int64)
    return pred, img_r

In [108]:
@torch.no_grad()
def save_gt_pred_cnn(pair_list, k, cnn_model, out_path):
    img_path, mask_path = pair_list[k]
    img  = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path)
    img_r  = img.resize((RESIZE[1], RESIZE[0]), resample=Image.BILINEAR)
    mask_r = mask.resize((RESIZE[1], RESIZE[0]), resample=Image.NEAREST)
    gt = ID_TO_TRAINID[np.array(mask_r, dtype=np.uint8)].astype(np.int64)
    pred, _ = predict_cnn_single(img, cnn_model)
    plt.figure(figsize=(14,4))
    plt.subplot(1,3,1); plt.title("Image"); plt.imshow(img_r); plt.axis("off")
    plt.subplot(1,3,2); plt.title("GT"); plt.imshow(colorize_trainid(gt)); plt.axis("off")
    plt.subplot(1,3,3); plt.title("CNN Pred"); plt.imshow(colorize_trainid(pred)); plt.axis("off")
    plt.tight_layout()
    out_path = Path(out_path)
    plt.savefig(out_path, dpi=200); plt.close()
    print("Saved:", out_path)

In [109]:
@torch.no_grad()
def save_gt_pred_sf(pair_list, k, sf_model, out_path):
    img_path, mask_path = pair_list[k]
    img  = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path)
    img_r  = img.resize((RESIZE[1], RESIZE[0]), resample=Image.BILINEAR)
    mask_r = mask.resize((RESIZE[1], RESIZE[0]), resample=Image.NEAREST)
    gt = ID_TO_TRAINID[np.array(mask_r, dtype=np.uint8)].astype(np.int64)
    pred, _ = predict_sf_single(img, sf_model)
    plt.figure(figsize=(14,4))
    plt.subplot(1,3,1); plt.title("Image"); plt.imshow(img_r); plt.axis("off")
    plt.subplot(1,3,2); plt.title("GT"); plt.imshow(colorize_trainid(gt)); plt.axis("off")
    plt.subplot(1,3,3); plt.title("SegFormer Pred"); plt.imshow(colorize_trainid(pred)); plt.axis("off")
    plt.tight_layout()
    out_path = Path(out_path)
    plt.savefig(out_path, dpi=200); plt.close()
    print("Saved:", out_path)

In [110]:
@torch.no_grad()
def save_gt_pred_compare(pair_list, k, cnn_model, sf_model, out_path):
    img_path, mask_path = pair_list[k]
    img  = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path)
    img_r  = img.resize((RESIZE[1], RESIZE[0]), resample=Image.BILINEAR)
    mask_r = mask.resize((RESIZE[1], RESIZE[0]), resample=Image.NEAREST)
    gt = ID_TO_TRAINID[np.array(mask_r, dtype=np.uint8)].astype(np.int64)
    pred_cnn, _ = predict_cnn_single(img, cnn_model)
    pred_sf,  _ = predict_sf_single(img, sf_model)

    plt.figure(figsize=(18,4))
    plt.subplot(1,4,1); plt.title("Image"); plt.imshow(img_r); plt.axis("off")
    plt.subplot(1,4,2); plt.title("GT"); plt.imshow(colorize_trainid(gt)); plt.axis("off")
    plt.subplot(1,4,3); plt.title("CNN Pred"); plt.imshow(colorize_trainid(pred_cnn)); plt.axis("off")
    plt.subplot(1,4,4); plt.title("SegFormer Pred"); plt.imshow(colorize_trainid(pred_sf)); plt.axis("off")
    plt.tight_layout()
    out_path = Path(out_path)
    plt.savefig(out_path, dpi=200); plt.close()
    print("Saved:", out_path)

In [111]:
def stage_visual_cnn():
    cnn = load_best_cnn_100()
    cnn.eval()

    rng = np.random.RandomState(VIS_RANDOM_SEED)
    idxs = rng.choice(len(val_pairs), size=min(N_VIS_SAMPLES, len(val_pairs)), replace=False)

    for j, k in enumerate(idxs):
        outp = FIG_DIR / f"val_cnn_gt_vs_pred_{j}_k{k}.png"
        save_gt_pred_cnn(val_pairs, int(k), cnn, outp)

    mark_done("vis_cnn_done", True)

In [112]:
def stage_visual_sf():
    sf = load_best_sf_100()
    rng = np.random.RandomState(VIS_RANDOM_SEED)
    idxs = rng.choice(len(val_pairs), size=min(N_VIS_SAMPLES, len(val_pairs)), replace=False)
    for j, k in enumerate(idxs):
        outp = FIG_DIR / f"val_sf_gt_vs_pred_{j}_k{k}.png"
        save_gt_pred_sf(val_pairs, int(k), sf, outp)
    mark_done("vis_sf_done", True)

In [113]:
def stage_visual_compare():
    cnn_ckpt = CKPT_DIR / "best_cnn_pct100.pth"
    sf_ckpt  = CKPT_DIR / "best_sf_pct100.pth"
    if (not cnn_ckpt.exists()) or (not sf_ckpt.exists()):
        print("[WARN] Necesitas ambos checkpoints pct100 para visual comparativa.")
        return
    cnn = load_best_cnn_100()
    sf  = load_best_sf_100()
    rng = np.random.RandomState(VIS_RANDOM_SEED)
    idxs = rng.choice(len(val_pairs), size=min(N_VIS_SAMPLES, len(val_pairs)), replace=False)
    for j, k in enumerate(idxs):
        outp = FIG_DIR / f"val_compare_gt_vs_pred_{j}_k{k}.png"
        save_gt_pred_compare(val_pairs, int(k), cnn, sf, outp)
    mark_done("vis_compare_done", True)

# Apply Corruptions

In [115]:
def stage_corruptions_cnn():
    cnn = load_best_cnn_100()

    clean_loader = make_loader_cnn(val_pairs, train=False)
    clean_miou, clean_acc, _ = evaluate_cnn(cnn, clean_loader)

    rows = []
    for corr, sev in CORRUPTIONS:
        loader = make_loader_cnn(val_pairs, train=False, corruption=corr, severity=sev)
        miou, acc, _ = evaluate_cnn(cnn, loader)
        rows.append({
            "corruption": corr, "severity": sev,
            "miou": miou, "acc": acc,
            "delta_miou": clean_miou - miou,
            "delta_acc": clean_acc - acc
        })
        print(f"[CNN {corr} s{sev}] mIoU={miou:.4f} Δ={clean_miou-miou:.4f}")

    out_path = TABLE_DIR / "robustness_cnn.csv"
    pd.DataFrame(rows).to_csv(out_path, index=False)
    print("Saved:", out_path)
    mark_done("corr_cnn_done", True)

In [116]:
def stage_corruptions_sf():
    sf = load_best_sf_100()

    clean_loader = make_loader_sf(val_pairs, train=False)
    clean_miou, clean_acc, _ = evaluate_sf(sf, clean_loader)

    rows = []
    for corr, sev in CORRUPTIONS:
        loader = make_loader_sf(val_pairs, train=False, corruption=corr, severity=sev)
        miou, acc, _ = evaluate_sf(sf, loader)
        rows.append({
            "corruption": corr, "severity": sev,
            "miou": miou, "acc": acc,
            "delta_miou": clean_miou - miou,
            "delta_acc": clean_acc - acc
        })
        print(f"[SF {corr} s{sev}] mIoU={miou:.4f} Δ={clean_miou-miou:.4f}")

    out_path = TABLE_DIR / "robustness_sf.csv"
    pd.DataFrame(rows).to_csv(out_path, index=False)
    print("Saved:", out_path)
    mark_done("corr_sf_done", True)

In [117]:
def stage_corruptions_compare():
    cnn_csv = TABLE_DIR / "robustness_cnn.csv"
    sf_csv  = TABLE_DIR / "robustness_sf.csv"
    if (not cnn_csv.exists()) or (not sf_csv.exists()):
        print("[WARN] Necesitas robustness_cnn.csv y robustness_sf.csv para comparar.")
        return
    d1 = pd.read_csv(cnn_csv).rename(columns={"miou":"cnn_miou","acc":"cnn_acc","delta_miou":"cnn_delta_miou","delta_acc":"cnn_delta_acc"})
    d2 = pd.read_csv(sf_csv).rename(columns={"miou":"sf_miou","acc":"sf_acc","delta_miou":"sf_delta_miou","delta_acc":"sf_delta_acc"})
    df = d1.merge(d2, on=["corruption","severity"])
    out_path = TABLE_DIR / "robustness_compare.csv"
    df.to_csv(out_path, index=False)
    print("Saved:", out_path)
    mark_done("corr_compare_done", True)

# Orchestator

In [119]:
def run(stages=None):
    """
    stages (examples):
      - ["train_cnn"]         -> only CNN training (all pcts)
      - ["train_sf"]          -> only SegFormer training (all pcts)
      - ["curve_cnn"]         -> only CNN curve
      - ["curve_sf"]          -> only SF curve
      - ["curve_compare"]     -> comparative curve (needs both)
      - ["eff_cnn"]           -> CNN efficiency
      - ["eff_sf"]            -> SF efficiency
      - ["eff_compare"]       -> compare efficiency (needs both)
      - ["vis_cnn"]           -> CNN GT vs Pred
      - ["vis_sf"]            -> SF GT vs Pred
      - ["vis_compare"]       -> combined (needs both)
      - ["corr_cnn"]          -> corruptions CNN
      - ["corr_sf"]           -> corruptions SF
      - ["corr_compare"]      -> compare corruptions (needs both)
      - ["all_cnn"]           -> CNN-only full pipeline
      - ["all_sf"]            -> SF-only full pipeline
      - ["all_compare"]       -> only comparisons (requires both done)
      - ["all"]               -> everything (cnn then sf then comparisons)
    """
    if stages is None:
        stages = ["all"]

    if "all" in stages:
        stages = [
            "train_cnn","curve_cnn","eff_cnn","vis_cnn","corr_cnn",
            "train_sf","curve_sf","eff_sf","vis_sf","corr_sf",
            "curve_compare","eff_compare","vis_compare","corr_compare"
        ]

    if "all_cnn" in stages:
        stages = ["train_cnn","curve_cnn","eff_cnn","vis_cnn","corr_cnn"]

    if "all_sf" in stages:
        stages = ["train_sf","curve_sf","eff_sf","vis_sf","corr_sf"]

    if "all_compare" in stages:
        stages = ["curve_compare","eff_compare","vis_compare","corr_compare"]

    # --- CNN pipeline
    if "train_cnn" in stages:
        stage_train_cnn_all_pcts()
    if "curve_cnn" in stages:
        stage_plot_curve_cnn()
    if "eff_cnn" in stages:
        stage_efficiency_cnn()
    if "vis_cnn" in stages:
        stage_visual_cnn()
    if "corr_cnn" in stages:
        stage_corruptions_cnn()

    # --- SegFormer pipeline
    if "train_sf" in stages:
        stage_train_sf_all_pcts()
    if "curve_sf" in stages:
        stage_plot_curve_sf()
    if "eff_sf" in stages:
        stage_efficiency_sf()
    if "vis_sf" in stages:
        stage_visual_sf()
    if "corr_sf" in stages:
        stage_corruptions_sf()

    # --- Comparative
    if "curve_compare" in stages:
        stage_plot_curve_compare()
    if "eff_compare" in stages:
        stage_efficiency_compare()
    if "vis_compare" in stages:
        stage_visual_compare()
    if "corr_compare" in stages:
        stage_corruptions_compare()

    if device.type == "cuda":
        torch.cuda.empty_cache()

    print("DONE. Outputs in:", OUTPUT_DIR)

# Test de mapeo

In [121]:
print(test_pairs[0])
print(Path(test_pairs[0][0]).exists(), Path(test_pairs[0][1]).exists())

('C:\\Users\\ALLAN TURING\\.cache\\kagglehub\\datasets\\electraawais\\cityscape-dataset\\versions\\2\\Cityscape Dataset\\leftImg8bit\\test\\berlin\\berlin_000000_000019_leftImg8bit.png', 'C:\\Users\\ALLAN TURING\\.cache\\kagglehub\\datasets\\electraawais\\cityscape-dataset\\versions\\2\\Fine Annotations\\gtFine\\test\\berlin\\berlin_000000_000019_gtFine_labelIds.png')
True True


In [122]:
img_path, mask_path = test_pairs[0]
mask = Image.open(mask_path)
mask_r = mask.resize((RESIZE[1], RESIZE[0]), resample=Image.NEAREST)
gt_raw = np.array(mask_r, dtype=np.uint8)
gt = ID_TO_TRAINID[gt_raw].astype(np.int64)

print("raw unique (first 30):", np.unique(gt_raw)[:30])
print("trainId unique:", np.unique(gt))
print("ignore ratio:", (gt == 255).mean())


raw unique (first 30): [0 1 3]
trainId unique: [255]
ignore ratio: 1.0


In [123]:
for lid in [7,8,11,24,26,33]:
    print(lid, "->", int(ID_TO_TRAINID[lid]))

7 -> 0
8 -> 1
11 -> 2
24 -> 11
26 -> 13
33 -> 18


In [124]:
pairs = val_pairs  # usa val primero (recomendado)
img_path, mask_path = pairs[0]

mask = Image.open(mask_path)
mask_r = mask.resize((RESIZE[1], RESIZE[0]), resample=Image.NEAREST)

gt_raw = np.array(mask_r, dtype=np.uint8)
gt = ID_TO_TRAINID[gt_raw].astype(np.int64)

print("mask file:", mask_path)
print("raw unique (first 30):", np.unique(gt_raw)[:30])
print("trainId unique:", np.unique(gt))
print("ignore ratio:", float((gt == 255).mean()))

mask file: C:\Users\ALLAN TURING\.cache\kagglehub\datasets\electraawais\cityscape-dataset\versions\2\Fine Annotations\gtFine\val\frankfurt\frankfurt_000000_000294_gtFine_labelIds.png
raw unique (first 30): [ 1  2  3  4  7  8 11 13 17 20 21 23 24 26]
trainId unique: [  0   1   2   4   5   7   8  10  11  13 255]
ignore ratio: 0.11572837829589844


In [125]:
cnn = load_best_cnn_100()
cnn.eval()

loader = make_loader_cnn(val_pairs, train=False)
imgs, masks = next(iter(loader))

with torch.no_grad():
    out = cnn(imgs.to(device))
logits = out["out"] if isinstance(out, dict) else out

print("imgs:", imgs.shape)
print("logits:", logits.shape)  # debe ser [B, 19, H, W]
print("NUM_CLASSES:", NUM_CLASSES)


imgs: torch.Size([4, 3, 512, 1024])
logits: torch.Size([4, 19, 512, 1024])
NUM_CLASSES: 19


In [126]:
cnn = load_best_cnn_100()
val_loader = make_loader_cnn(val_pairs, train=False)
miou, acc, _ = evaluate_cnn(cnn, val_loader)
print("CLEAN VAL:", miou, acc)

CLEAN VAL: 0.7217209339141846 0.9528476710590452


In [127]:
cnn = load_best_cnn_100()
val_loader = make_loader_cnn(test_pairs, train=False)
miou, acc, _ = evaluate_cnn(cnn, val_loader)
print("CLEAN VAL:", miou, acc)

CLEAN VAL: 0.0 0.0


> Note: Robustness experiments are reported on the Cityscapes validation split, since the official test set is intended for server-side evaluation.

# Main

In [130]:
if __name__ == "__main__":
    # Examples:
    # run(["train_cnn"])  -> only baseline, resumable
    # run(["train_sf"])   -> only segformer, resumable
    # run(["all_cnn"])    -> full CNN pipeline only
    # run(["all_sf"])     -> full SF pipeline only
    # run(["all_compare"]) -> only comparative outputs (needs both)
    run(["all"])         # -> all stages
    # run(["corr_cnn", "all_sf", "all_compare"])2

CNN DATA EFFICIENCY: pct=10 | train_samples=297
[SKIP] CNN pct=10 ya entrenado. CKPT: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\checkpoints\best_cnn_pct10.pth
Saved/Updated: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\tables\cnn_data_efficiency_summary.csv
CNN DATA EFFICIENCY: pct=25 | train_samples=743
[SKIP] CNN pct=25 ya entrenado. CKPT: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\checkpoints\best_cnn_pct25.pth
Saved/Updated: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\tables\cnn_data_efficiency_summary.csv
CNN DATA EFFICIENCY: pct=50 | train_samples=1487
[SKIP] CNN pct=50 ya entrenado. CKPT: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\checkpoints\best_cnn_pct50.pth
Saved/Updated: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\tables\cnn_data_efficiency_summary.csv
CNN DATA EFFICIENCY: pct=100 | train_samples=2975
[SKIP] CNN pct=100 ya entrenado. CKPT: C:\Users\ALLAN TURI

SF Epoch 1/30:   0%|          | 0/148 [00:00<?, ?it/s]

[SF pct=10] Epoch 1/30 | loss=0.1381 | val mIoU=0.6311 | acc=0.9371 (best)


SF Epoch 2/30:   0%|          | 0/148 [00:00<?, ?it/s]

[SF pct=10] Epoch 2/30 | loss=0.1251 | val mIoU=0.6369 | acc=0.9370 (best)


...

[SF pct=100] Epoch 17/30 | loss=0.1035 | val mIoU=0.6654 | acc=0.9421 


SF Epoch 18/30:   0%|          | 0/1487 [00:00<?, ?it/s]

[SF pct=100] Epoch 18/30 | loss=0.1027 | val mIoU=0.6640 | acc=0.9423 


SF Epoch 19/30:   0%|          | 0/1487 [00:00<?, ?it/s]

[SF pct=100] Epoch 19/30 | loss=0.1015 | val mIoU=0.6692 | acc=0.9429 
[SF pct=100] Early stopping.
Saved/Updated: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\tables\sf_data_efficiency_summary.csv
Saved: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\figures\data_efficiency_curve_sf.png
Saved: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\tables\efficiency_sf.csv
Saved: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\figures\val_sf_gt_vs_pred_0_k195.png
Saved: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\figures\val_sf_gt_vs_pred_1_k417.png
Saved: C:\Users\ALLAN TURING\Documents\ProyectoTE_AyalaMacias\outputs\figures\val_sf_gt_vs_pred_2_k478.png
[SF gaussian_blur s3] mIoU=0.6076 Δ=0.0645
[SF motion_blur s3] mIoU=0.5976 Δ=0.0746
[SF gaussian_noise s3] mIoU=0.4984 Δ=0.1737
[SF brightness s3] mIoU=0.6641 Δ=0.0080
[SF contrast s3] mIoU=0.6673 Δ=0.0048
[SF jpeg s3] mIoU=0.5498 Δ=0.1223
Saved: C:\Users\ALLAN